[View in Colaboratory](https://colab.research.google.com/github/JozeeLin/google-tensorflow-exercise/blob/master/%E7%BB%8F%E5%85%B8%E5%8D%B7%E7%A7%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C_ResNet.ipynb)

## ResNet(Residual Neural Network)

ResNet通过使用Residual Unit成功训练152层深的神经网络。ResNet的结构可以极快地加速超深神经网络的训练，模型的准确率也有非常大的提升。

**注意：Highway NetWork原理与ResNet很相似。Highway NetWork的目标就是解决极深的神经网络难以训练的问题。**

**Highway NetWork相当于修改了每一层的激活函数，此前的激活函数只是对输入做一个非线性变换，HN则允许保留一定比例的原始输入x。**

**这样前面一层的信息，有一定比例可以不经过矩阵乘法和非线性变换，直接传输到下一层，仿佛一条信息高速公路，因此得名HN**

**HN 主要通过gating units学习如何控制网络中的信息流，即学习原始信息应保留的比例。这个可学习的gating机制，
借鉴于LSTM循环神经网络中的gating，使之可以使用梯度下降算法来训练。并可以配合多种非线性激活函数，学习极深的神经网络现在变得可行了。**



In [0]:
import collections
import tensorflow as tf
import time
import math
from datetime import datetime
slim = tf.contrib.slim

In [0]:
# 使用collections.namedtuple设计ResNet基本Block模块组的named tuple，并用它创建Block的类，但只包含数据结构，不包含具体方法。
class Block(collections.namedtuple('Block',['scope','unit_fn','args'])):
  'A named tuple describing a ResNet block'

In [0]:
#降采样subsample的方法
def subsample(inputs, factor, scope=None):
  if factor == 1:
    return inputs
  else:
    return slim.max_pool2d(inputs,[1,1],stride=factor,scope=scope)

In [0]:
#用于创建卷积层
def conv2d_same(inputs, num_outputs, kernel_size, stride, scope=None):
  if stride == 1:
    return slim.conv2d(inputs, num_outputs, kernel_size, stride=1, padding='SAME', scope=scope)
  else:
    pad_total = kernel_size - 1
    pad_beg = pad_total // 2
    pad_end = pad_total - pad_beg
    inputs = tf.pad(inputs, [[0,0], [pad_beg, pad_end],
                            [pad_beg,pad_end],[0,0]])
    return slim.conv2d(inputs, num_outputs, kernel_size, stride=stride, padding='VALID', scope=scope)
  

In [0]:
#定义堆叠Blocks函数
@slim.add_arg_scope
def stack_blocks_dense(net, blocks,outputs_collections=None):
  for block in blocks:
    with tf.variable_scope(block.scope, 'block',[net]) as sc:
      for i, unit in enumerate(block.args):
        with tf.variable_scope('unit_%d' % (i+1), values=[net]):
          unit_depth, unit_depth_bottleneck, unit_stride=unit
          net = block.unit_fn(net, 
                             depth=unit_depth,
                             depth_bottleneck=unit_depth_bottleneck,
                             stride=unit_stride)
      net = slim.utils.collect_named_outputs(outputs_collections, sc.name, net)
  return net

In [0]:
#创建通用的arg_scope,其功能是用来定义某些函数的参数默认值
def resnet_arg_scope(is_training=True,
                    weight_decay=0.0001,
                    batch_norm_decay=0.997,
                    batch_norm_epsilon=1e-5,
                    batch_norm_scale=True
                    ):
  batch_norm_params={
      'is_training':is_training,
      'decay':batch_norm_decay,
      'epsilon':batch_norm_epsilon,
      'scale':batch_norm_scale,
      'updates_collections':tf.GraphKeys.UPDATE_OPS,
  }
  
  with slim.arg_scope(
      [slim.conv2d],
      weights_regularizer=slim.l2_regularizer(weight_decay),
      weights_initializer=slim.variance_scaling_initializer(),
      activation_fn=tf.nn.relu,
      normalizer_fn=slim.batch_norm,
      normalizer_params=batch_norm_params
  ):
    with slim.arg_scope([slim.batch_norm], **batch_norm_params):
      with slim.arg_scope([slim.max_pool2d], padding='SAME') as arg_sc:
        return arg_sc

In [0]:
#bottleneck 残差学习单元，论文中提到的full preactivation residual unit的一个变种。
#主要区别有两点:
#一是在每一层前都用了batch normalizer，二是对输入进行preactivation，而不是在卷积进行激活函数处理
@slim.add_arg_scope
def bottleneck(inputs, depth, depth_bottleneck, stride, outputs_collections=None, scope=None):
  with tf.variable_scope(scope,'bottleneck_v2', [inputs]) as sc:
    depth_in = slim.utils.last_dimension(inputs.get_shape(), min_rank=4)
    preact = slim.batch_norm(inputs, activation_fn=tf.nn.relu, scope='preact')
    
    if depth == depth_in:
      shortcut = subsample(inputs, stride, 'shortcut')
    else:
      shortcut = slim.conv2d(preact, depth, [1,1], stride=stride, 
                            normalizer_fn=None, activation_fn=None,
                            scope='shortcut')
      
    residual = slim.conv2d(preact, depth_bottleneck, [1,1], stride=1, scope='conv1')
    residual = conv2d_same(residual, depth_bottleneck, 3, stride, scope='conv2')
    residual = slim.conv2d(residual, depth, [1,1], stride=1, normalizer_fn=None, activation_fn=None,scope='conv3')
    
    output = shortcut + residual
    return slim.utils.collect_named_outputs(outputs_collections, sc.name, output)

In [0]:
#定义生成ResNet V2的主函数，只要预先定义好网络的残差学习模块组blocks
def resnet_v2(inputs, blocks,
             num_classes=None,
             global_pool=True,
             include_root_block=True,
             reuse=None,
             scope=None):
  with tf.variable_scope(scope, 'resnet_v2', [inputs], reuse=reuse) as sc:
    
    end_points_collection = sc.original_name_scope+'_end_points'
    
    with slim.arg_scope([slim.conv2d,bottleneck,stack_blocks_dense], 
                       outputs_collections=end_points_collection):
      net = inputs
      if include_root_block:
        with slim.arg_scope([slim.conv2d], activation_fn=None,normalizer_fn=None):
          net = conv2d_same(net, 64, 7, stride=2, scope='conv1')
        net = slim.max_pool2d(net, [3,3], stride=2, scope='pool1')
        
      net = stack_blocks_dense(net, blocks)
      
      net = slim.batch_norm(net, activation_fn=tf.nn.relu, scope='postnorm')
      if global_pool:
        net = tf.reduce_mean(net, [1,2], name='pool4', keep_dims=True) #实现全局平均池化，效率比直接用avg_pool高
        
      if num_classes is not None:
        net = slim.conv2d(net, num_classes, [1,1], activation_fn=None, normalizer_fn=None, scope='logits') #添加输出通道
      
      #再添加一个softmax层输出网络结果
      end_points = slim.utils.convert_collection_to_dict(end_points_collection)
      if num_classes is not None:
        end_points['predictions'] = slim.softmax(net, scope='predictions') 
         
      return net, end_points

### 设置几个不同深度的ResNet网络配置，来设计层数分别为50，101，152和200的ResNet。

In [0]:
#50层的ResNet，4个残差学习Blocks的units数量分别为3，4，6和3，总层数为(3+4+6+3)x3+2=50.需要注意的是，
#残差学习模块之前的卷积、池化已经将尺寸缩小了4倍，前3个blocks又都包含步长为2的层，因此总尺寸缩小了4x8=32倍，输入图片尺寸最后变为224/32=7。
# 和Inception V3类似，ResNet不断使用步长为2的层来缩减尺寸，但同时输出通道数也在持续增加，最后达到了2048
def resnet_v2_50(intpus,
                num_classes=None,
                global_pool=True,
                reuse=None,
                scope='resnet_v2_50'):
  blocks=[
      Block('block1', bottleneck, [(256, 64,1)]*2+[(256,64,2)]),
      Block('block2', bottleneck, [(512,128,1)]*3+[(512,128,2)]),
      Block('block3', bottleneck, [(1024,256,1)]*5+[1024,245,2]),
      Block('block4', bottleneck, [(2048,512,1)]*3)
  ]
  
  return resnet_v2(inputs, blocks, num_classes, global_pool, include_root_block=True, reuse=reuse, scope=scope)

In [0]:
#101层ResNet和50层相比，主要变化就是把4个Blocks的units数量从3，4，6，3，提升到了3，4，23，3.即将第三个残差学习block的units数增加到接近4倍

def resnet_v2_101(intpus,
                 num_classes=None,
                 global_pool=True,
                 reuse=None,
                 scope='resnet_v2_101'):
  blocks=[
      Block('block1', bottleneck, [(256,64,1)]*2+[(256,64,2)]),
      Block('block2', bottelnect, [(512,128,1)]*3+[(512,128,2)]),
      Block('block3', bottleneck, [(1024,256,1)]*22+[(1024,256,2)]),
      Block('block4', bottleneck, [(2048,512,1)]*3)
  ]
  
  return resnet_v2(inputs, blocks, num_classes, global_pool, include_root_block=True, reuse=reuse, scope=scope)

In [0]:
#152层的ResNet，则是将第二个Block的unit数提高到8，将第三个Block的unit提高到36。3,8,36,3
def resnet_v2_152(inputs, num_classes=None,
                 global_pool=True,
                 reuse=None,
                 scope='resnet_v2_152'):
  blocks=[
      Block('block1', bottleneck, [(256,64,1)]*2+[(256,64,2)]),
      Block('block2', bottleneck, [(512,128,1)]*7+[(512,128,2)]),
      Block('block3', bottleneck, [(1024,256,1)]*35+[(1024,256,2)]),
      Block('block4', bottleneck, [(2048,512,1)]*3)
  ]
  
  return resnet_v2(inputs, blocks, num_classes, global_pool, include_root_block=True, reuse=reuse, scope=scope)

In [0]:
#200层ResNet，相比152层的ResNet，没有继续提升第三个Block的units数，而是将第二个Block的unit数一下子提升到了3,23,36,3
def resnet_v2_200(inputs, num_classes=None,global_pool=True,
                 reuse=None,
                 scope='resnet_v2_200'):
  blocks = [
      Block('block1', bottleneck, [(256,64,1)]*2+[(256,64,2)]),
      Block('block2', bottleneck, [(512,128,1)]*22+[(512,128,2)]),
      Block('block3', bottleneck, [(1024,256,1)]*35+[(1024,256,2)]),
      Block('block4', bottleneck, [(2048, 512,1)]*3)
  ]
  
  return resnet_v2(inputs, blocks, num_classes, global_pool, include_root_block=True, reuse=reuse, scope=scope)

In [0]:
def time_tensorflow_run(session, target, info_string):
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0

    for i in range(num_batches + num_steps_burn_in):
      start_time = time.time()
      _ = session.run(target)
      duration = time.time() - start_time
      if i >= num_steps_burn_in:
        if not i % 10:
          print ('%s: step %d, duration = %.3f' % (datetime.now(), i - num_steps_burn_in, duration))
          
        total_duration += duration
        total_duration_squared += duration * duration
          
    mn = total_duration / num_batches
    vr = total_duration_squared / num_batches - mn * mn
    sd = math.sqrt(vr)
    print ('%s: %s across %d steps, %.3f +/- %.3f sec / batch' % (datetime.now(), info_string, num_batches, mn, sd))

In [14]:
#最后使用一直以来的评测函数time_tensorflow_run来测试152层深的ResNet的forward性能。
batch_size = 32
height,width = 224,224
inputs = tf.random_uniform((batch_size, height, width, 3))
with slim.arg_scope(resnet_arg_scope(is_training=False)):
  net, end_points = resnet_v2_152(inputs, 1000)
  
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
num_batches = 100
time_tensorflow_run(sess, net, 'forward')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
2018-05-06 09:52:35.813644: step 0, duration = 0.474
2018-05-06 09:52:40.576186: step 10, duration = 0.479
2018-05-06 09:52:45.367559: step 20, duration = 0.480
2018-05-06 09:52:50.158952: step 30, duration = 0.476
2018-05-06 09:52:54.960180: step 40, duration = 0.479
2018-05-06 09:52:59.757984: step 50, duration = 0.480
2018-05-06 09:53:04.556299: step 60, duration = 0.479
2018-05-06 09:53:09.349402: step 70, duration = 0.478
2018-05-06 09:53:14.141642: step 80, duration = 0.479
2018-05-06 09:53:19.059099: step 90, duration = 0.485
2018-05-06 09:53:23.416910: forward across 100 steps, 0.481 +/- 0.009 sec / batch
